# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [115]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()
data.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**

1) В таблице 12 столбцов и 21525 строк;
2) В столбцах days_employed и total_income есть пропущенные значения;
3) В названиях столбцов нет лишних символов.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [116]:
#берем значения по модулю, избавляемся от отрицательных значений.
data['days_employed'] = data['days_employed'].abs() 
#заменяем пропущенные значения стажа работы медианным значением, т.к среднее значение не превышает медианное в несколько раз
#data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
#тоже самое проделываем для столбца 'total_income'
#data['total_income'] = data['total_income'].fillna(data['total_income'].median())
#data[['days_employed','total_income']].head(15)

#------------------------------------------------после ревью 2 -----------------------------------------------------------------
# я поразмышлял над данными в таблице и подумал, стоит ли вообще пропуски в количественных переменных, заполнять числовыми
# данными вообще, в частности медианой или средним, мы же не знаем откуда пропуски взялись, что если этот человек вообще не
# работал или системная ошибка при выгрузке. Если заполним медианой и средним, то это повысит шанс получить кредит.
# а на данных по сути не отразиться.
# я решил просто заполнить их нулями и посмотреть на итоговые выводы.
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

**Вывод**

Избавились от отрицательных и нулевых значений

### Замена типа данных

In [117]:
#для столбцов 'days_employed' и 'total_income' точность в миллион знаков не нужна, поэтому вещественный тип данных 
#можно перевести в целые числа
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

#---------------------------------------------------- после ревью ------------------------------------------------------------
# посчитаем стаж в годах и добавим столбец
data['years_employed'] = data['days_employed'] / 365
#изменим тип данных на целочисленный
data['years_employed'] = data['years_employed'].astype('int')
#выразим значение total_income в тыс.рубл
data['total_income_thousand'] = data['total_income'] / 1000
data['total_income_thousand'] = data['total_income_thousand'].astype('int')
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_thousand
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,253
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,112
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,145
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,267
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,158


**Вывод**

1) Для столбцов 'days_employed' и 'total_income' точность в миллион знаков не нужна
2) Таблица преобретает более приятный вид
3) В столбце days_employed стаж работы превышает возраст пенсионера, что может повлиять на результаты, в рамках задания нет требования это исправлять, поэтому оставлю как есть.
4) В столбце education регистр отличается, нужно привести к общему

### Обработка дубликатов

In [118]:
#посчитаем количество дубликатов в таблице и посмотрим на них
from collections import Counter
#Counter(data['children'])
# переводим антиребенка в ребенка :)
data['children'] = data['children'].abs()
# обрабатываем чрезмерно многодетных
children_med = data[data['children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_med)

# приводим education к общему регистру
data['education'] = data['education'].str.lower()
# проверяем столбец income_type на дубликаты
data['income_type'].value_counts()
# проверяем столбец family_status на дубликаты
data['family_status'].value_counts()
# проверяем столбец family_status на дубликаты
data['gender'].value_counts()
# находим строку с не понятным полом, на итоговый результат это не повлияет, можем и оставить.
# находи сумму дуплиткатов
data.duplicated().sum()
# удаляем дубликаты
data = data.drop_duplicates().reset_index(drop=True)
#проверяем
data.duplicated().sum()

0

очистили таблицу от дубликатов 

### Лемматизация

In [126]:
from pymystem3 import Mystem
m=Mystem()
#напишем функцию которая будет возвращать список лемм и запишем их в новый столбец
def lemma(row):
    lemma = m.lemmatize(row)
    lemma = ''.join(lemma).strip()
    return lemma
data['lemmatized_purpose'] = data['purpose'].apply(lemma)
data.head(13)
#data['lemmatized_purpose'].value_counts()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_thousand,lemmatized_purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,253,покупка жилье,высокий,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,112,приобретение автомобиль,средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,145,покупка жилье,средний,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,267,дополнительный образование,высокий,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,158,сыграть свадьба,высокий,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2,255,покупка жилье,высокий,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7,240,операция с жилье,высокий,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,135,образование,средний,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18,95,на проведение свадьба,средний,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,144,покупка жилье для семья,средний,недвижимость


**Вывод**

Выделили леммы из столбца 'purpose'. Предположи что будет 4 категори целей кредита Жилье(недвижимость), образование, свадьба, автомобиль.

### Категоризация данных

**Вывод**

In [120]:
#столбец total_income слишком разнородный, выделим категории в зависимости от уровня дохода и запишем их в новый столбец
#по данным росстата средний доход в России в 2020 году составляет 35361 руб округлим до 35000 и напишем функцию которая будет 
#присваивать категории
#https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D1%85%D0%BE%D0%B4%D1%8B_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#:~:text=%D0%9F%D0%BE%20%D0%BE%D1%84%D0%B8%D1%86%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%BC%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%20%D0%BE%D1%82%20%D0%A0%D0%BE%D1%81%D1%81%D1%82%D0%B0%D1%82%D0%B0,14%20986%20%D1%80%D1%83%D0%B1%D0%BB%D0%B5%D0%B9%20(%24208).


# изменим функцию для категоризации данных по уровню дохода 
def total_income_category(value):
    if value < 50:
        return 'низкий'
    if 50 < value < 150:
        return 'средний'
    else:
        return 'высокий'
data['total_income_category'] = data['total_income_thousand'].apply(total_income_category)


#разобъем цели кредита на категории 
estate = ['недвижимость', 'жилье']
wedding = ['свадьба']
auto = ['автомобиль']
education = ['образование']

# напишем функцию
data['purpose_category'] = 0
def category(list_of_words, category):
    words = '|'.join(list_of_words)
    index = data[data['lemmatized_purpose'].str.contains(words)].index.to_list()
    for i in index:
        data.loc[i, 'purpose_category'] = category
    return data
category(wedding, 'свадьба')
category(estate, 'недвижимость')
category(auto, 'автомобиль')
category(education, 'образование')
Counter(data['purpose_category'])


Counter({'недвижимость': 10811,
         'автомобиль': 4306,
         'образование': 4013,
         'свадьба': 2324})

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [121]:
#создадим сводную таблицу с даннымы 
debt_from_children = pd.DataFrame()
#добавим в таблицу столбец с количеством элементов в группе должников
debt_from_children['count_children_debt'] = data.groupby('children')['debt'].count()
#посчитаем сумму должников
debt_from_children['sum_children_debt'] = data.groupby('children')['debt'].sum()
#найдем отношение суммы к количеству
#----------------------------------------------- после ревью -------------------------------------------------------------------------
debt_from_children['result_children_debt'] = debt_from_children['sum_children_debt'] / debt_from_children['count_children_debt'] * 100
#-------------------------------------------------------------------------------------------------------------------------------------
#отсортируем по убыванию
debt_from_children.sort_values('result_children_debt', ascending = False)

,count_children_debt,sum_children_debt,result_children_debt
children,,,
4,41,4,9.756098
2,2052,194,9.454191
1,4855,445,9.165808
3,330,27,8.181818
0,14167,1071,7.559822
5,9,0,0.000000


**Вывод**

Результаты не однозначны. С увеличением кол-ва детей, увеличивается задолжность по кредиту. Бездетные реже имееют задолжность, чем люди хотя бы с одним ребенком. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [122]:
#проводим аналогичные действия, чтобы выяснить зависимость на основе семейного статуса.
debt_from_family_status = pd.DataFrame()
debt_from_family_status['count_family_status'] = data.groupby('family_status')['debt'].count()
debt_from_family_status['sum_family_status'] = data.groupby('family_status')['debt'].sum()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] * 100 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,count_family_status,sum_family_status,result_family_status
family_status,,,
Не женат / не замужем,2810,274,9.750890
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343


**Вывод**

Люди, не зарегистрировшие брак, чаще имеют задожность по кредитам. Разведенные и овдовевшие реже других имеют задолжность.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [123]:
#посчитаем конверсию
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = data.groupby('total_income_category')['debt'].sum()
debt_from_total_income['count'] = data.groupby('total_income_category')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count'] * 100
debt_from_total_income.sort_values('conversion', ascending = False)

,sum,count,conversion
total_income_category,,,
средний,819,9759,8.392253
высокий,729,9220,7.906725
низкий,193,2475,7.797980


**Вывод**

<font color='blue'>Люди со средним доходом чаще имеют задолжности по кредитам</font>



- Как разные цели кредита влияют на его возврат в срок?

In [124]:
#посмотрим как цели кредита влияют на наличие задолжности
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] * 100 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
автомобиль,403,4306,9.359034
образование,370,4013,9.220035
свадьба,186,2324,8.003442
недвижимость,782,10811,7.233373


**Вывод**

Чаще всего имеют задолжность, люди, взявшие кредит на автомобиль и образование. Меньше всего должников в категории недвижимость.


## Шаг 4. Общий вывод

1)С увеличением кол-ва детей, увеличивается задолжность по кредиту. Бездетные реже имееют задолжность, чем люди хотя бы с одним ребенком.
2)Люди, не зарегистрировшие брак, чаще имеют задожность по кредитам. Разведенные и овдовевшие реже других имеют задолжность.
3)<font color='blue'>Люди со средним доходом чаще имеют задолжности по кредитам.</font>
4)Чаще всего имеют задолжность, люди, взявшие кредит на автомобиль и образование. Меньше всего должников в категории недвижимость. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.